<a href="https://colab.research.google.com/github/UtsavRao/PRODIGY_DS_04/blob/main/TASK_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization
import pandas
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import LearningRateScheduler
import numpy as np
from tensorflow.keras.callbacks import ReduceLROnPlateau
from google.colab import drive
from tensorflow.keras import regularizers

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
data_reddit = pandas.read_csv("Reddit_Data.csv")
data_twitter = pandas.read_csv("Twitter_Data.csv")
data_reddit.rename(columns = {'clean_comment': 'text'}, inplace = True)
data_twitter.rename(columns = {'clean_text': 'text'}, inplace = True)

data = pandas.concat([data_reddit, data_twitter], ignore_index = True)
data = pandas.DataFrame(data)

In [4]:
data.dropna(axis = 0, inplace = True)

In [5]:
def preprocess_text(text):
    if isinstance(text, str):  # Check if the text is a non-null string
        # Convert text to lowercase
        text = text.lower()

        # Tokenization
        tokens = word_tokenize(text)

        # Remove punctuation and non-alphanumeric tokens
        tokens = [token for token in tokens if token.isalnum()]

        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words]

        # Join tokens back into a sentence
        preprocessed_text = ' '.join(tokens)

        return preprocessed_text
    else:
        return ""  # Return an empty string for NaN or non-string values

def tokenize(item):
  tokenizer = Tokenizer(oov_token="<OOV>")
  tokenizer.fit_on_texts(item)
  sequences = tokenizer.texts_to_sequences(item)
  max_length = 65  # Choose an appropriate value
  padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')
  vocab_size = len(tokenizer.word_index) + 1
  return padded_sequences, vocab_size

def lr_schedule(epoch, lr):
    return lr * np.exp(1)

In [6]:
ordinal_mapping = {-1: 0, 0: 1, 1: 2}

# Apply ordinal encoding to your labels using the apply function
data['encoded_sentiment'] = data['category'].apply(lambda x: ordinal_mapping[x])
data['preprocessed_text'] = data['text'].apply(preprocess_text)

In [7]:
x = data.pop("preprocessed_text")
y = data.pop('encoded_sentiment')
x, vocab_size = tokenize(x)
y = to_categorical(y, 3)

X_train,X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=42)

X_train = tf.convert_to_tensor(X_train)
y_train = tf.convert_to_tensor(y_train)
X_test = tf.convert_to_tensor(X_test)
y_test = tf.convert_to_tensor(y_test)

In [8]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.6, patience=0, verbose=1, min_lr=1e-6)

embedding_dim = 65
max_length = 65

In [9]:
tf.random.set_seed(42)
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(LSTM(8, return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(rate=0.4))
model.add(LSTM(8, return_sequences=True))
model.add(Dropout(rate=0.4))
model.add(LSTM(8, return_sequences=True))
model.add(Dropout(rate=0.2))
model.add(LSTM(8))
model.add(Dropout(rate=0.2))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.004), metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=25, validation_split=0.2, callbacks=[early_stopping,lr_scheduler])

Epoch 1/25
1299/1299 [==============================] - 258s 188ms/step - loss: 0.6804 - accuracy: 0.7231 - val_loss: 0.5168 - val_accuracy: 0.8299 - lr: 0.0040
Epoch 2/25
1299/1299 [==============================] - 211s 163ms/step - loss: 0.4402 - accuracy: 0.8593 - val_loss: 0.4484 - val_accuracy: 0.8501 - lr: 0.0040
Epoch 3/25
1299/1299 [==============================] - 205s 158ms/step - loss: 0.3767 - accuracy: 0.8909 - val_loss: 0.4481 - val_accuracy: 0.8588 - lr: 0.0040
Epoch 4/25
1299/1299 [==============================] - ETA: 0s - loss: 0.3374 - accuracy: 0.9093
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.002400000113993883.
1299/1299 [==============================] - 204s 157ms/step - loss: 0.3374 - accuracy: 0.9093 - val_loss: 0.4676 - val_accuracy: 0.8593 - lr: 0.0040
Epoch 5/25
1299/1299 [==============================] - 204s 157ms/step - loss: 0.2662 - accuracy: 0.9321 - val_loss: 0.4269 - val_accuracy: 0.8685 - lr: 0.0024
Epoch 6/25
1299/1299 [===========

KeyboardInterrupt: ignored

In [10]:
model.evaluate(X_test,y_test)

287/287 [==============================] - 6s 21ms/step - loss: 0.4696 - accuracy: 0.8728


[0.4695638418197632, 0.8727629780769348]